# MaskGIT Demo

## Clone Repository

In [ ]:
is_colab = True
if is_colab:
    !git clone https://github.com/valeoai/Halton-MaskGIT.git
    %cd Halton-MaskGIT
    %pip install omegaconf>=2.0.0 einops>=0.3.0 webdataset>=2.0 huggingface_hub clean-fid torch-fidelity torchmetrics

## Download Pretrained models

In [ ]:
if is_colab:
    from huggingface_hub import hf_hub_download
    hf_hub_download(repo_id="FoundationVision/LlamaGen",  filename="vq_ds16_c2i.pt",  local_dir="./saved_networks")
    hf_hub_download(repo_id="llvictorll/Halton-Maskgit", filename="ImageNet_384_large.pth", local_dir="./saved_networks")

In [ ]:
import random
import matplotlib.pyplot as plt
import torch
import torchvision.utils as vutils
from huggingface_hub import hf_hub_download

from Utils.utils import load_args_from_file
from Utils.viz import show_images_grid
from Trainer.cls_trainer import MaskGIT
from Sampler.halton_sampler import HaltonSampler
from Sampler.confidence_sampler import ConfidenceSampler

# MaskGIT initialisation

In [ ]:
config_path = "Config/base_cls2img.yaml"        # Path to your config file
args = load_args_from_file(config_path)
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

maskgit = MaskGIT(args)

In [ ]:
def viz(x, nrow=10, pad=2, size=(18, 18)):
    """
    Visualize a grid of images.

    Args:
        x (torch.Tensor): Input images to visualize.
        nrow (int): Number of images in each row of the grid.
        pad (int): Padding between the images in the grid.
        size (tuple): Size of the visualization figure.

    """
    nb_img = len(x)
    min_norm = x.min(-1)[0].min(-1)[0].min(-1)[0].view(-1, 1, 1, 1)
    max_norm = x.max(-1)[0].max(-1)[0].max(-1)[0].view(-1, 1, 1, 1)
    x = (x - min_norm) / (max_norm - min_norm)

    x = vutils.make_grid(x.float().cpu(), nrow=nrow, padding=pad, normalize=False)
    plt.figure(figsize = size)
    plt.axis('off')
    plt.imshow(x.permute(1, 2, 0))
    plt.show()

def decoding_viz(gen_code, mask, maskgit):
    """
    Visualize the decoding process of generated images with associated masks.

    Args:
        gen_code (torch.Tensor): Generated code for decoding.
        mask (torch.Tensor): Mask used for decoding.
        maskgit (MaskGIT): MaskGIT instance.
    """
    start = torch.FloatTensor([1, 1, 1]).view(1, 3, 1, 1).expand(1, 3, maskgit.input_size, maskgit.input_size) * 0.8
    end = torch.FloatTensor([0.01953125, 0.30078125, 0.08203125]).view(1, 3, 1, 1).expand(1, 3, maskgit.input_size, maskgit.input_size) * 1.4
    code = torch.stack((gen_code), dim=0).squeeze()
    mask = torch.stack((mask), dim=0).view(-1, 1, maskgit.input_size, maskgit.input_size).cpu()

    with torch.no_grad():
        x = maskgit.ae.decode_code(torch.clamp(code, 0, maskgit.args.mask_value))

    binary_mask = (1-mask) * start + mask * end
    binary_mask = vutils.make_grid(binary_mask, nrow=len(gen_code), padding=1, pad_value=0.4, normalize=False)
    binary_mask = binary_mask.permute(1, 2, 0)

    plt.figure(figsize = (18, 2))
    plt.gca().invert_yaxis()
    plt.pcolormesh(binary_mask, edgecolors='w', linewidth=.5)
    plt.axis('off')
    plt.show()

    viz(x, nrow=len(gen_code))

# MaskGIT Sampling With the Halton Scheduler

In [ ]:
sm_temp_min = 1.1   # Minimum softmax temperature for sampling.
sm_temp_max = 1.1   # Maximum softmax temperature for sampling.
top_k=-1            # If > 0, applies top-k sampling for token selection. 
temp_pow = 1        # Exponent for temperature scheduling.
w=2                 # CFG weight
sched_pow=2         # Power factor for the progressive unmasking schedule.
step=32             # Number of steps to sample an image
temp_warmup=1       # Number of initial steps where temperature is reduced.
randomize=True      # If True, applies random shifts to the Halton sequence for diverse sampling.

sampler = HaltonSampler(sm_temp_min=sm_temp_min, sm_temp_max=sm_temp_max, temp_pow=temp_pow, temp_warmup=temp_warmup, w=w, sched_pow=sched_pow, step=step, randomize=randomize, top_k=top_k)
# goldfish, chicken, tiger cat, hourglass, ship, dog, race car, airliner, teddy bear
labels = torch.LongTensor([1, 7, 282, 604, 724, 179, 681, 850]).to(args.device)

# Generate sample
gen_sample, gen_code, l_mask = sampler(maskgit, nb_sample=len(labels), labels=labels)
viz(gen_sample, nrow=10, size=(18, 18))

In [ ]:
nb_class = 8
nb_sample = 2
nb_row = 8
# Generate all ImageNet Classes
for l in range(0, 1000, nb_class):
    labels = [l+i for i in range(nb_class)] * nb_sample
    labels = torch.LongTensor(labels).to(args.device)
    # Generate sample
    gen_sample, gen_code, l_mask = sampler(maskgit, nb_sample=len(labels), labels=labels)
    x = viz(gen_sample, nrow=nb_row, size=(18, 18))